In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from netgen.csg import Pnt


In [ ]:
#help(unit_square)
netgen_mesh = unit_square.GenerateMesh(maxh=1.0)

mesh = Mesh(netgen_mesh)
#help(netgen_mesh)
unref_verts = mesh.vertices

unref_edges = mesh.edges

parent_el = mesh.ne
#issues with this-> maskes certain spaces not work
# mesh.Refine()
# mesh.Refine()
# mesh.Refine()
#other spaces dont work
H_h = HDiv(mesh,order=0,RT=True) #order_policy=ORDER_POLICY.CONSTANT
#might need to be number of faces (in unrefined mesh) because elements are have 1 face
unref_dofnr = H_h.ndof #mesh.ne
print(unref_dofnr)
#Draw(mesh)

In [ ]:


mesh.ngmesh.SplitPowellSabin()
mesh.ngmesh.Update()

#mesh._updateBuffers()

H_h.Update() 

H_h = Compress(H_h)


#might need to be number of faces in barycentrically refined mesh
bary_dofnr = H_h.ndof #mesh.ne
print(bary_dofnr)

#mesh._updateBuffers()


Draw(mesh)

In [ ]:

#help(H_h)

In [ ]:
#->assume if vertex with smallest nr has 5 edges that those are the 5 edges with smallest nr
# for e in unref_edges:
#     print(e,e.vertices)
#     # help(e)
#     # break
#     # for e in v.edges:
#     #     print(e)
#     #     # print(H_h.GetDofNrs(e))
#     #     # help(e)
#     #     # break


# for e in mesh.edges:
#     print(e,e.elements)

# print(mesh.nedge)
# print(H_h.ndof)


In [ ]:
for el in mesh.Elements():
    print(el.edges)
    #print(len(H_h.GetDofNrs(el)))
    #print(mesh.GetParentElement(el))
    # for v in el.vertices:
    #     print(v.nr)
    # print(el.edges)
    # print(el.faces)

In [ ]:
dofs_one_element = 3
element_per_dof = 14 #H_h.ndof//mesh.ne
# print(netgen_mesh.dim)
#bary_dofnr//unref_dofnr == 6 (for L2)
children = np.zeros((unref_dofnr,element_per_dof*dofs_one_element),dtype=np.int64) - 1

indices = np.zeros(unref_dofnr, dtype=np.int64)

occurences = np.zeros(bary_dofnr,dtype=np.int8)

for par in unref_verts:
    #print(el.nr)
    parent = par.nr
    for el in par.elements:
        for dofnr in H_h.GetDofNrs(el):
            children[parent,indices[parent]] = dofnr
            indices[parent] += 1
            occurences[dofnr] += 1
        #help(el)
        #break

print(children)


occurences[0:unref_dofnr] = 1

print(occurences)
#TODO Count up occurences of dofs in children list (in mapping matrix just increment corresponding entry according to some rule)

In [ ]:
# Setting the unused coarse dofs manually doesnt work, for some reason(?) they still dont activate

# print(H_h.ndof)
# print(mesh.nedge)
# #HDiv keeps unuesed dofs unless compressed
# print(H_h.FreeDofs())
# print(H_h.CouplingType(0))
# print(H_h.FreeDofs())
# help(H_h)
# #help(BilinearForm)
# #help(H_h.FreeDofs())

# for i in range(H_h.ndof):
#     if i < 13:
#         H_h.FreeDofs().Set(i)
#         H_h.SetCouplingType(i,COUPLING_TYPE.WIREBASKET_DOF)
#     else:
#         H_h.FreeDofs().Clear(i)
#         #H_h.SetCouplingType(i,COUPLING_TYPE.UNUSED_DOF)

# print(H_h.FreeDofs())

# #H_h = Compress(H_h)

# #H_h.Update()

# #H_h.UpdateDofTables()

# #H_h.FinalizeUpdate()

# print(H_h.FreeDofs())

# print(H_h.CouplingType(0))

# u,v = H_h.TnT()

# # b = BilinearForm(H_h,gucci=H_h.FreeDofs())

# # b += u * v * dx

# # b.Assemble()

# # print(b.mat.ToDense())

# # print(H_h.FreeDofs())

# #print(H_h.CouplingType(0))

In [ ]:
#H_h = L2(mesh,order=0) 

#help(gfu)

gfu = GridFunction(H_h,multidim = H_h.ndof, autoupdate=False)



In [ ]:
#gfu.Set(0) 

# for e in mesh.edges:
#     print(e.nr)
#     dofnr = H_h.GetDofNrs(e)[0]
#     print(dofnr)
        
for i in range(H_h.ndof):
    gfu.vecs[i][i] = 3/6


gfu.vecs[0][5] = -3/6

gfu.vecs[0][2] = 2/6

gfu.vecs[0][7] = -2/6

gfu.vecs[0][17] = -1/6

gfu.vecs[0][18] = 1/6



Draw(gfu, animate = False, min = 0, max = 2.0, deformation = False)

In [ ]:
netgen_mesh = unit_square.GenerateMesh(maxh=1)

ref_mesh = Mesh(netgen_mesh)

H_h = HDiv(ref_mesh,order=0,RT=True)

gfu = GridFunction(H_h,multidim = H_h.ndof)

#gfu.Set(0) 

for i in range(H_h.ndof):
    gfu.vecs[i][i] = 1.0

Draw(gfu, animate = False, min = 0, max = 2.0, deformation = False)

In [ ]:
netgen_mesh = unit_square.GenerateMesh(maxh=1)

netgen_mesh.SplitPowellSabin()

ref_mesh = Mesh(netgen_mesh)

H_h = HDiv(ref_mesh,order=0,RT=True)

gfu = GridFunction(H_h,multidim = H_h.ndof)

#gfu.Set(0) 

for i in range(H_h.ndof):
    gfu.vecs[i][i] = 1.0

Draw(gfu, animate = True, min = 0, max = 1.0, deformation = False)